![Imandrabot](../docs/images/kostya_ros_medium_1.png)

In this notebook we will go through creation and verification of a Robotic Operating System (ROS) node in Imandra. We will make an robot cotrol node that controls the motion of a simple 2-wheeler bot:

![Imandrabot](../docs/images/Imandrabot.png)

We'll create a controller that uses the laser scannner to intelligently avoid obstacles and drive around the scene. The Imandra ML code can be compiled in OCaml and plugged into the ROS system -- the behavior of the bot can be observed in the Gazebo simulator.    

Then we'll illustrate how to use Imandra to formally verify various statements about the model and how to find bugs and corner cases by exploring the Imandra-generated generated counterexamples for failed theorems.

# 1. ROS message OCaml types

For our Imandra-ROS project we’ve processed all the standard ROS messages with our code generation tool creating a collection of strongly-typed IML/OCaml bindings for them. But, in order to keep this notebook self-contained we'll define the necessary messaging modules here.  

First, we'll need to declare the message type that will control our robot. This is typically done with a `Twist` message from the `geometry_msgs` standard ROS package. We want to mimic ROS messaging nomenclauture as close as possible, so we'll create an OCaml/Imadra `module` with the same name as the package and will place the necessary type/message declaraions inside:

In [1]:
module Geometry_msgs = struct
  type vector3 = 
    { vector3_x : int
    ; vector3_y : int
    ; vector3_z : int
    }
  type twist = 
    { twist_linear  : vector3
    ; twist_angular : vector3
    }    
end

module Geometry_msgs :
  sig
    type vector3 = { vector3_x : int; vector3_y : int; vector3_z : int; }
    type twist = { twist_linear : vector3; twist_angular : vector3; }
  end


One thing that you might have noticed is that we've replaced floating point values for vector coordinates with `int`s. It is much easier for Imandra to reason about integers, so we assume that there is a common factor of 100000 that multiplying all the incoming floating point values and divides all the outgoing integers. (That effectively makes our unit of measurement to be 10 micrometers).

Let's move on and declare the incoming messages: 
 - `LaserScan` sensor input message from the `sensor_msgs` ROS package
 - and the `Clock` message from the `Rosgraph_msg` ROS package  
 
We define the wrapping modules for both messages and declare their datatypes:

In [2]:
module Sensor_msgs = struct
  type laserScan = 
    { laserScan_range_min : int 
    ; laserScan_range_max : int 
    ; laserScan_ranges : int list 
    }
end
module Rosgraph_msgs = struct
  type time = 
    { seconds     : int
    ; nanoseconds : int
    }
  type clock = { clock : time }
end

module Sensor_msgs :
  sig
    type laserScan = {
      laserScan_range_min : int;
      laserScan_range_max : int;
      laserScan_ranges : int list;
    }
  end
module Rosgraph_msgs :
  sig
    type time = { seconds : int; nanoseconds : int; }
    type clock = { clock : time; }
  end


# 2. Creating a simple ROS Node model

We want to create some simple but non-trivial robot controller that makes our bot drive around avoiding the obstacles. The bot is going to drive forward until one of the laser scanner ranges becomes too low, meaning that we've gotten too close to some obstacle -- in that case we want the bot to stop and turn until the scanner tells us that the road ahead is clear. To make model a bit more complex, we'd like to implement the ability to choose the turning direction depending on the laser scanner ranges.

One might try to make a "naiive" controller that doesn't have any memory about the previous bot states and observations and just reacts to the currently observed scanner values. This will quickly lead to the bot being "stuck" in infinite oscillatory loops. E.g. here is the bot that decides which side to turn depending on the first value in the `ranges` array:

![Imandrabot](../docs/images/Stuck.gif)

To avoid this kind of 

## 2.1 State datatype

Working with Imandra we’ve adopted a standard way to construct formal models of message-driven systems. At the top of the model we have a single OCaml datatype that holds all the data needed to describe the system at a given moment, including incoming and outgoing messages. We call this record type `state`. Together with this `state` type we define a `one_step` transition `state -> state` function, which performs a single logically isolated step of the simulation and returns the new `state` after the transition.

As an example, consider an IML/OCaml type declaration for a simple ROS node that is able to accept `rosgraph_msgs/Clock` and `sensor_msgs/LaserScan` standard ROS messages. We also want the state to store two values: 
 - the latest minimal value of the ranges that the laser sensor returns
 - the preferred side for the robot to turn -- either left or right. 
 
Finally, we want the node to be able to send `geometry_msgs/Twist` ROS message depending on the stored `min_range` data:

In [4]:
type incoming_msg = 
  | Clock  of Rosgraph_msgs.clock
  | Sensor of Sensor_msgs.laserScan

type outgoing_msg =
  | Twist of Geometry_msgs.twist

type direction = CW | CCW 
type mode = Driving | Turning 

type state =
  { mode : mode
  ; min_range : int option
  ; direction : direction option
  ; incoming  : incoming_msg option
  ; outgoing  : outgoing_msg option 
  }

type incoming_msg =
    Clock of Rosgraph_msgs.clock
  | Sensor of Sensor_msgs.laserScan
type outgoing_msg = Twist of Geometry_msgs.twist
type direction = CW | CCW
type mode = Driving | Turning
type state = {
  mode : mode;
  min_range : int option;
  direction : direction option;
  incoming : incoming_msg option;
  outgoing : outgoing_msg option;
}


## 2.2 State transition `one_step` function

To implement our node, we'll need a function that scans through a list of values and returns the minimum value *and its index*. We'll make a generic function `foldi` that does an indexed version of `List.fold_right`.   

Couple of things to notice here:
- the function has its "termination proof" -- that means that Imandra managed to prove that recusive calls in this function will not end up in an infinite loop. Imandra proves such things using inductive reasoning and is able to prove further statements about other properties of such functions.   
- instead of doing it like in the tail-recursive `fold_left`, we use `fold_right` approach instead. Having no accumulator, `fold_right` is generally more friendly for inductive proofs than `fold_left`.

In [5]:
let rec foldi ~base ?(i=0) f l =
  match l with
  | [] -> base
  | x :: tail -> f i x (foldi f ~base ~i:(i+1) tail)

val foldi : base:'a -> ?i:Z.t -> (Z.t -> 'b -> 'a -> 'a) -> 'b list -> 'a =
  <fun>


termination proof Termination proof call `foldi base (Some ((if Is_a(Some, *opt*) then Option.get *opt* else 0) + 1))
f_2 (List.tl l)` from `foldi base *opt* f_2 l` original foldi base *opt* f_2 l sub foldi base (Some ((if Is_a(Some, *opt*) then Option.get *opt* else 0) + 1))
f_2 (List.tl l) original ordinal Ordinal.Int (Ordinal.count l) sub ordinal Ordinal.Int (Ordinal.count (List.tl l)) path [not (l = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.008s details Expand smt_stats num checks 7 arith assert lower 7 arith pivots 5 rlimit count 1908 mk clause 3 datatype occurs check 43 mk bool var 54 arith assert upper 5 datatype splits 3 decisions 7 arith add rows 15 propagations 2 conflicts 7 arith fixed eqs 5 datatype accessor ax 5 arith conflicts 1 datatype constructor ax 8 num allocs 597543683 final checks 6 added eqs 34 del clause 1 arith eq adapter 4 memory 4.320000 max memory 16.160000 Expand start[0.008s]
 not (l = []) && Ordinal.count l >= 0 && Ordinal.count (List.tl l) >= 0
 ==> List.tl l = []
 || Ordinal.Int (Ordinal.count (List.tl l)) Ordinal.<<
 Ordinal.Int (Ordinal.count l) simplify into (not
 ((not (l = []) && Ordinal.count l >= 0) && Ordinal.count (List.tl l) >= 0)
 || List.tl l = [])
|| Ordinal.Int (Ordinal.count (List.tl l)) Ordinal.<<
 Ordinal.Int (Ordinal.count l) expansions [] rewrite_steps forward_chaining unroll expr (Ordinal.<<_131 (Ordinal.Int_122 (|count_`ty_1 list`_2228|
 (|get.… expansions unroll expr (|count_`ty_1 list`_2228| (|get.::.1_2206| l_2217)) expansions unroll expr (|count_`ty_1 list`_2228| l_2217) expansions unsat (let ((a!1 (ite (>= (|count_`ty_1 list`_2228| (|get.::.1_2206| l_2217)) 0)
 (|count_`…

Lets now create a simple `one_step` function for a ROS node that reacts to the incoming messages. We want the robot, on incoming `Clock` message, to either move forward or turn. On incoming `Scan` message, 


depending on the stored `min_range` value. To this end, we first create two helper functions: 
 - `get_min_range` extract the minimal value of the ranges in a `Scan` message 
 - `make_twist_message` creates a `Twist` message with the given forward and angular velocities

In [14]:
let get_min_and_direction msg =
  let max = msg.Sensor_msgs.laserScan_range_max in
  let lst = msg.Sensor_msgs.laserScan_ranges in
  let min_range = List.fold_right ~base:max
    (fun x a -> if x < a then x else a) lst in
  let mini = foldi ~base:(max, 0)
    (fun i a b -> if a < fst b then (a,i) else b) in
  let _ , idx = mini lst in
  if idx < List.length lst / 2 then min_range, CW else min_range, CCW
  
let make_twist_message v omega=
  let open Geometry_msgs in
  let mkvector x y z = { vector3_x = x; vector3_y = y; vector3_z = z } in 
  Twist { twist_linear  = mkvector v 0 0
        ; twist_angular = mkvector 0 0 omega 
        } 

val get_min_and_direction : Sensor_msgs.laserScan -> int * direction = <fun>
val make_twist_message : int -> int -> outgoing_msg = <fun>


Upon receiving the `Clock` message, we want to just either drive forward or turn in place, depending on the values stored in the state: if we are in the `Driving` mode -- we dirve forward, if we are in the `Turning` mode, then we turn either left of right, depending on the stored value of the `direction` field.

In [15]:
let process_clock_message state =
  match state.mode with 
  | Driving -> { state with outgoing = Some (make_twist_message 10000 0) } 
  | Turning -> begin
    match state.direction with 
    | None
    | Some ( CW ) -> { state with outgoing = Some (make_twist_message 0   10000) } 
    | Some (CCW ) -> { state with outgoing = Some (make_twist_message 0 (-10000))} 
  end

val process_clock_message : state -> state = <fun>


When receiving the `LaserScan` message, we are updating the variables on our `state`:  
  - In the `Dirving` mode we check if the scanned minimal range is less than 0.2m. If that is the case we transition to the `Turning` mode; otherwise we just keep driving.
  - In the `Turning` mode we check if the scanned minimal range is greater than 0.25m -- in that case we switch to the `Driving` mode. If we are still in the `Turning` mode -- we update the stored minimal range variable and the turning direction depending on whether then new value is less than the stored value.

In [16]:
let process_sensor_message state min_range min_direction =
  match state.mode , state.min_range with 
  | Driving , _    ->
    if min_range < 20000 then 
      { state with 
        mode      = Turning
      ; direction = Some min_direction
      ; min_range = Some min_range 
      }
    else
      { state with mode = Driving; min_range = None; direction = None } 
  | Turning , None -> 
    if min_range > 25000 then 
      { state with mode = Driving; min_range = None; direction = None } 
    else
      { state with 
        direction = Some min_direction
      ; min_range = Some min_range
      } 
  | Turning , Some old_range -> 
    if min_range > 25000 then 
      { state with mode = Driving; min_range = None; direction = None } 
    else if min_range > old_range then state else 
    { state with 
      direction = Some min_direction
    ; min_range = Some min_range
    }  

val process_sensor_message : state -> int -> direction -> state = <fun>


With the help of these functions, we can create our `one_step` transition function, which just dispathces the messages to the appropriate helper function above.

In [17]:
let one_step state =
  match state.incoming with None -> state | Some in_msg ->
  let state = { state with incoming = None; outgoing = None } in
  match in_msg with 
  | Sensor laserScan -> 
    let min_range, min_direction = get_min_and_direction laserScan in
    process_sensor_message state min_range min_direction
  | Clock  _ -> 
    process_clock_message state

val one_step : state -> state = <fun>


## 2.3 Running the model as a ROS node

One can write the model into an OCaml code file:   
https://github.com/AestheticIntegration/imandra-ros/blob/master/imandra_model/src-model/ros_model.ml  
and then compile it into an executable node, using our ROS node wrapper. 

Here is the model, controling our "imandrabot" in the Gazebo simulation environment:

![Imandrabot](../docs/images/Imandra_Demo.gif)

# 3. Verifying the ROS node model

## 3.1 Verifying outgoing `Twist` message at `Clock` ticks 

Our model is designed in such a way that it updates its state parameters upon `LaserScan` messages 


$$ \forall s. IsClock(IncomingMessage(s)) \,\Rightarrow\, IsTwist(OutgoingMessage(s)) $$

Meaning that for every state $s$, if the state contains an incoming message and this message is a `Clock` message, then the state's `outgoing` message is a `Twist` after we've called `one_step` on it.  

We can almost literally encode this formal expression as an Imandra `theorem`:

In [18]:
let is_clock msg = match msg with  Some ( Clock _ ) -> true | _ -> false ;;
let is_twist msg = match msg with  Some ( Twist _ ) -> true | _ -> false ;;

theorem clock_creates_outbound state =
  is_clock state.incoming ==> is_twist (one_step state).outgoing

val is_clock : incoming_msg option -> bool = <fun>
val is_twist : outgoing_msg option -> bool = <fun>
val clock_creates_outbound : state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.007s details Expand smt_stats num checks 2 arith assert lower 2 rlimit count 1813 mk clause 39 mk bool var 216 arith assert upper 2 datatype splits 28 decisions 17 propagations 35 conflicts 10 datatype accessor ax 48 datatype constructor ax 33 num allocs 680859693 added eqs 293 del clause 3 arith eq adapter 1 memory 11.310000 max memory 16.160000 Expand start[0.007s]
 (if Is_a(Clock, Option.get :var_0:.incoming)
 && Is_a(Some, :var_0:.incoming)
 then true else false)
 ==> (if Is_a(Twist,
 Option.get
 (if :var_0:.incoming = None then :var_0: else …).outgoing)
 && Is_a(Some,
 (if :var_0:.incoming = None then :var_0: else …).outgoing)
 then true else false) simplify into not
(Is_a(Clock, Option.get :var_0:.incoming) && Is_a(Some, :var_0:.incoming))
|| Is_a(Twist,
 Option.get (if :var_0:.incoming = None then :var_0: else …).outgoing)
 && Is_a(Some, (if :var_0:.incoming = None then :var_0: else …).outgoing) expansions [] rewrite_steps forward_chaining unsat (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_2533 (get.Some.0_2535 (inco… digraph "proof" {
p_213 [label="Start ((if Is_a(Clock, Option.get :var_0:.incoming)\l && Is_a(Some, :var_0:.incoming)\l then true else false)\l ==\> (if Is_a(Twist,\l Option.get\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l && Is_a(Some,\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l then true else false)\l :time 0.007s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_213 -> p_212 [label=""];
p_212 [label="Simplify (not\l (Is_a(Clock, Option.get :var_0:.incoming)\l && Is_a(Some, :var_0:.incoming))\l \|\| Is_a(Twist,\l Option.get\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l && Is_a(Some,\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_212 -> p_211 [label=""];
p_211 -> p_z3_1042 [label="",fontname="courier",fontsize=14];
p_z3_1042 [label="(let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_2533 (get.Some.0_2535 (inco…",shape=box,fontname="courier",fontsize=14];p_211 [label="unsat",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading.. digraph "call graph" {
goal [label="not\l(not (Is_a(Clock, Option.get state.incoming) && Is_a(Some, state.incoming))\l \|\| Is_a(Twist,\l Option.get (if state.incoming = None then state else …).outgoing)\l && Is_a(Some, (if state.incoming = None then state else …).outgoing))",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_183 [label="foldi ((Destruct(Sensor, 0, …)).Sensor_msgs.laserScan_range_max, 0) None\lanon_fun.get_min_and_direction.1\l(Destruct(Sensor, 0, …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_183 [label="calls"];
call_187 [label="List.fold_right anon_fun.get_min_and_direction.0\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_187 [label="calls"];
call_178 [label="List.length\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_178 [label="calls"];
}
 Load graph Loading..

One can see that Imandra says that it "Proved" the theorem, meaning that Imandra has formally checked that this property holds for all possible input states. 

## 3.2 Veryfying stopping if close to a wall

Lets try to prove that if all the ranges in the incoming laser scan message are less than 0.2m then we definitely transition to the `Turning` state. We'll try to encode it like this:

In [53]:
verify ( fun state ->
  let open Sensor_msgs in
  match state.incoming with None | Some (Clock _ ) -> true 
  | Some ( Sensor data ) ->
  (  List.for_all (fun x -> x < 20000) data.laserScan_ranges    
  ) ==> (one_step state).mode = Turning
)

- : state -> bool = <fun>
module CX : sig val state : state end


We have failed to prove the statement and Imandra have created a counterexample `CX` module for us. This module  contains concrete values for the parameters that violate the statement of the theorem. Examining the state we notice that the incoming `laserScan_ranges` list is empty. 

In [54]:
CX.state

- : state =
{mode = Turning; min_range = Some 7719; direction = None;
 incoming =
  Some
   (Sensor
     {Sensor_msgs.laserScan_range_min = 3; laserScan_range_max = 25001;
      laserScan_ranges = []});
 outgoing = None}


Adding the extra requirement that the list is not `[]`, we successfully verify the statement:

In [55]:
verify ( fun state ->
  let open Sensor_msgs in
  match state.incoming with None | Some (Clock _ ) -> true 
  | Some ( Sensor data ) ->
  (  data.laserScan_ranges <> []
  && List.for_all (fun x -> x < 20000) data.laserScan_ranges    
  ) ==> (one_step state).mode = Turning
)

- : state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 4 definitions 0 inductions 0 search_time 0.016s details Expand smt_stats arith offset eqs 4 num checks 10 arith assert lower 40 arith pivots 22 rlimit count 7508 mk clause 86 datatype occurs check 1584 mk bool var 539 arith assert upper 42 datatype splits 51 decisions 339 arith add rows 29 propagations 314 interface eqs 1 conflicts 31 arith fixed eqs 9 datatype accessor ax 57 minimized lits 2 arith conflicts 3 arith assert diseq 8 datatype constructor ax 259 final checks 13 added eqs 1321 del clause 36 arith eq adapter 20 memory 43.480000 max memory 64.530000 num allocs 83411433924.000000 Expand start[0.016s]
 if :var_0:.incoming = None
 || Is_a(Some, :var_0:.incoming)
 && Is_a(Clock, Option.get :var_0:.incoming)
 then true
 else
 not
 ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 = [])
 && List.for_all anon_fun._verify_target.0
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 ==> (if :var_0:.incoming = None then :var_0:
 else if Is_a(Sensor, Option.get :var_0:.incoming) then … else …).mode
 = Turning simplify into ((:var_0:.incoming = None
 || Is_a(Some, :var_0:.incoming) && Is_a(Clock, Option.get :var_0:.incoming))
 || not
 (not
 ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 = [])
 && List.for_all anon_fun._verify_target.0
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges))
|| (if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Driving then … else …).mode
 = Turning expansions [] rewrite_steps forward_chaining unroll expr (let ((a!1 (Sensor_msgs.laserScan_range_max_2143
 (get.Sensor.0_5562 (get.Some.0_5564 (i… expansions unroll expr (let ((a!1 (Sensor_msgs.laserScan_range_max_2143
 (get.Sensor.0_5562 (get.Some.0_5564 (i… expansions unroll expr (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_5562 (get.Some.0_5564 (inco… expansions unroll expr (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_5562 (get.Some.0_5564 (inco… expansions unsat (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_5562 (get.Some.0_5564 (inco… digraph "proof" {
p_629 [label="Start (if :var_0:.incoming = None\l \|\| Is_a(Some, :var_0:.incoming)\l && Is_a(Clock, Option.get :var_0:.incoming)\l then true\l else\l not\l ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l = [])\l && List.for_all anon_fun._verify_target.0\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l ==\> (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming) then … else …).mode\l = Turning\l :time 0.016s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_629 -> p_628 [label=""];
p_628 [label="Simplify (((:var_0:.incoming = None\l \|\| Is_a(Some, :var_0:.incoming)\l && Is_a(Clock, Option.get :var_0:.incoming))\l \|\| not\l (not\l ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l = [])\l && List.for_all anon_fun._verify_target.0\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges))\l \|\| (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if :var_0:.mode = Driving then … else …\l else if :var_0:.mode = Driving then … else …).mode\l = Turning\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_628 -> p_627 [label=""];
p_627 [label="Unroll ([List.fold_right anon_fun.get_min_and_direction.0\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges],\l [[]])",shape=box,style=filled,fontname="courier",fontsize=14];
p_627 -> p_626 [label=""];
p_626 [label="Unroll ([foldi\l ((Destruct(Sensor, 0, …)).Sensor_msgs.laserSc

## 3.3 Verifying initialization of `min_range` value

The `min_range` state variable is an `option` and initially is `None`. Lets verify that upon receiving a `sensor_msgs/LaserScan` message, the resulting state stores some `min_range` value (not `None`). 

$$ \forall s. NonEmptyLaserScan(IncomingMessage(s)) \,\Rightarrow\, MinValue(one\_step(s)) \ne None $$

Meaning that for every state $s$, if the state contains an incoming message and this message is a `LaserScan` that contains non-empty list of ranges, then the state's `min_value` is not `None` after we've called `one_step` on it.  

Again, we encode this formal expression in Imandra:

In [63]:
verify ( fun state ->
  let open Sensor_msgs in
  match state.incoming with None | Some (Clock _ ) -> true 
  | Some ( Sensor data ) ->
  (  data.laserScan_ranges <> []
  ) ==> (one_step state).min_range <> None
)

- : state -> bool = <fun>
module CX : sig val state : state end


Again, we have failed to prove it. Examining the incoming state we notice that the `laserScan_ranges` list contains a value that is larger than 20000. 

In [64]:
CX.state

- : state =
{mode = Turning; min_range = Some 34998; direction = None;
 incoming =
  Some
   (Sensor
     {Sensor_msgs.laserScan_range_min = 3; laserScan_range_max = 33856;
      laserScan_ranges = [36293]});
 outgoing = None}


Adding the constraint that the incoming ranges are less than 20000, we prove the statement: 

In [70]:
verify ( fun state ->
  let open Sensor_msgs in
  match state.incoming with None | Some (Clock _ ) -> true 
  | Some ( Sensor data ) ->
  (  data.laserScan_ranges <> []
  && List.for_all (fun x -> x < 20000) data.laserScan_ranges    
  ) ==> (one_step state).min_range <> None
)

- : state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 4 definitions 0 inductions 0 search_time 0.017s details Expand smt_stats num checks 10 arith assert lower 34 arith pivots 17 rlimit count 7284 mk clause 76 datatype occurs check 1582 mk bool var 519 arith assert upper 31 datatype splits 51 decisions 297 arith add rows 18 propagations 337 interface eqs 2 conflicts 34 arith fixed eqs 10 datatype accessor ax 51 minimized lits 4 arith conflicts 3 arith assert diseq 5 datatype constructor ax 265 final checks 14 added eqs 1365 del clause 34 arith eq adapter 14 memory 52.610000 max memory 74.010000 num allocs 192565042228.000000 Expand start[0.017s]
 if :var_0:.incoming = None
 || Is_a(Some, :var_0:.incoming)
 && Is_a(Clock, Option.get :var_0:.incoming)
 then true
 else
 not
 ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 = [])
 && List.for_all anon_fun._verify_target.0
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 ==> not
 ((if :var_0:.incoming = None then :var_0:
 else if Is_a(Sensor, Option.get :var_0:.incoming) then … else …).min_range
 = None) simplify into ((:var_0:.incoming = None
 || Is_a(Some, :var_0:.incoming) && Is_a(Clock, Option.get :var_0:.incoming))
 || not
 (not
 ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 = [])
 && List.for_all anon_fun._verify_target.0
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges))
|| not
 ((if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Driving then … else …).min_range
 = None) expansions [] rewrite_steps forward_chaining unroll expr (let ((a!1 (Sensor_msgs.laserScan_range_max_2143
 (get.Sensor.0_6969 (get.Some.0_6971 (i… expansions unroll expr (let ((a!1 (Sensor_msgs.laserScan_range_max_2143
 (get.Sensor.0_6969 (get.Some.0_6971 (i… expansions unroll expr (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_6969 (get.Some.0_6971 (inco… expansions unroll expr (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_6969 (get.Some.0_6971 (inco… expansions unsat (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_6969 (get.Some.0_6971 (inco… digraph "proof" {
p_850 [label="Start (if :var_0:.incoming = None\l \|\| Is_a(Some, :var_0:.incoming)\l && Is_a(Clock, Option.get :var_0:.incoming)\l then true\l else\l not\l ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l = [])\l && List.for_all anon_fun._verify_target.0\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l ==\> not\l ((if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming) then … else …).min_range\l = None)\l :time 0.017s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_850 -> p_849 [label=""];
p_849 [label="Simplify (((:var_0:.incoming = None\l \|\| Is_a(Some, :var_0:.incoming)\l && Is_a(Clock, Option.get :var_0:.incoming))\l \|\| not\l (not\l ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l = [])\l && List.for_all anon_fun._verify_target.0\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges))\l \|\| not\l ((if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if :var_0:.mode = Driving then … else …\l else if :var_0:.mode = Driving then … else …).min_range\l = None)\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_849 -> p_848 [label=""];
p_848 [label="Unroll ([List.fold_right anon_fun.get_min_and_direction.0\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges],\l [[]])",shape=box,style=filled,fontname="courier",fontsize=14];
p_848 -> p_847 [label=""];
p_847 [label="Unroll ([foldi\l ((Destruct(Sensor, 0, …)